In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00


In [5]:
import pandas as pd
from datasets import load_dataset, Dataset
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset
import torch
import spacy


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import spacy.cli
spacy.cli.download("en_core_web_md")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


Это датасет, содержащий вопросы и ответы касательно ментального здоровья. В итоге получится некий ChatBot Therapist🤗

In [7]:
dataset = load_dataset("Amod/mental_health_counseling_conversations")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
data = pd.DataFrame(dataset['train'])
data_dict = data.to_dict(orient='records')

In [11]:
nlp = spacy.load("en_core_web_md")

В качестве трансформера, я буду использвать модель, котору уже обучала на этом датасете

In [14]:
model_name = "/content/drive/MyDrive/dioloGPT_mental_health_model"
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/dioloGPT_mental_health_tok')
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
model_config = model.config
model_config.pad_token_id = tokenizer.eos_token_id

In [15]:
vectors = []
for item in data_dict:
    doc = nlp(item["Context"])
    vectors.append((doc, item["Response"]))

Как я определила гибридную модель: сначала ответ чатбота определяется с помошью spacy similarity, и если значение similarity больше 0.9, то выводится именно этот ответ. Если similarity меньше, то ответ генерирует обученная модел. Таким образом пользователь будет получать ответы быстрее (так как для генерации ответа моделью требуется больше времени, когда не каждый ответ генерируется моделью, пользователь быстрее получает ответ на свой вопрос)

In [27]:
while True:
    user_input = input("You say: ")
    if user_input.lower() == 'exit':
        print ('Exiting the bot')
        break
    else:

        user_doc = nlp(user_input)
        similarities = [user_doc.similarity(question) for question, _ in vectors]

        if max(similarities) > 0.9:
            index_of_most_similar = similarities.index(max(similarities))
            most_similar_answer = vectors[index_of_most_similar][1]
            print("ChatBot (Spacy Similarity):", most_similar_answer)

        else:
          tokenized_input = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True)
          input_ids = tokenized_input["input_ids"]
          attention_mask = tokenized_input["attention_mask"]


          output = model.generate(input_ids, attention_mask=attention_mask, max_length=1000, num_beams=5, no_repeat_ngram_size=2, do_sample=True, temperature=0.2)
          generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
          generated_response = generated_response.replace(user_input, '').strip()

          print("ChatBot (GPT):", generated_response)

You say: Hi! i had a huge fight with a friend and now i feel sad and anxious. What should i do?
ChatBot (GPT): Hi there, Thank you for your question. I'm very sorry that you are feeling sad. It's normal to feel that way. First, I want to say that I am so glad you reached out. There are a lot of different types of anxiety that can be experienced during an anxiety attack. First, is it possible that there is something that is keeping you from reaching out? If so, try to talk to your friend about it and see if they would be willing to discuss it with you. If it is possible, consider seeing a therapist who specializes in anxiety disorders. They may be able to help you figure out where the anxiety is coming from and what is contributing to the feeling of sadness. And, of course, there are many different ways that anxiety can affect one's self-esteem, so I would encourage you to look at each of these ways in which it might be helpful to reach out for help.I hope this was helpful. Be well.Robi